In [ ]:
!pip install poetry
!pip install pathlib
!pip install typing
!pip install implicit
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 46.1 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.1.0
    Uninstalling platformdirs-4.1.0:
      Successfully uninstalled platformdirs-4.1.0
  Attempting uninstall: keyring
    Found existing installation: keyring 23.5.0
    Uninstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 2.3 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
from typing import Tuple, List
import pandas as pd
import implicit
import scipy

def load_user_artists(user_artists_file: Path) -> scipy.sparse.csr_matrix:
    """Load the user artists file and return a user-artists matrix in csr
    fromat.
    """
    user_artist = pd.read_csv("/content/sample_data/user_artists.dat", sep="\t")
    user_artist.set_index(["userID", "artistID"], inplace=True)
    coo = scipy.sparse.coo_matrix(
        (
            user_artist.weight.astype(float),
            (
                user_artist.index.get_level_values(0),
                user_artist.index.get_level_values(1),
            ),
        )
    )
    return coo.tocsr()

class ArtistRetriever:
    """The ArtistRetriever class gets the artist name from the artist ID."""

    def __init__(self):
        self._artists_df = None

    def get_artist_name_from_id(self, artist_id: int) -> str:
        """Return the artist name from the artist ID."""
        return self._artists_df.loc[artist_id, "name"]

    def load_artists(self, artists_file: Path) -> None:
        """Load the artists file and stores it as a Pandas dataframe in a
        private attribute.
        """
        artists_df = pd.read_csv("/content/sample_data/artists.dat", sep="\t")
        artists_df = artists_df.set_index("id")
        self._artists_df = artists_df


class ImplicitRecommender:
    """The ImplicitRecommender class computes recommendations for a given user
    using the implicit library.

    Attributes:
        - artist_retriever: an ArtistRetriever instance
        - implicit_model: an implicit model
    """

    def __init__(
        self,
        artist_retriever: ArtistRetriever,
        implicit_model: implicit.recommender_base.RecommenderBase,
    ):
        self.artist_retriever = artist_retriever
        self.implicit_model = implicit_model

    def fit(self, user_artists_matrix: scipy.sparse.csr_matrix):
        """Fit the model to the user artists matrix."""
        self.implicit_model.fit(load_user_artists(Path("/content/sample_data/user_artists.dat")))

    def recommend(self, user_id: int, user_artists_matrix: scipy.sparse.csr_matrix, n = 3):
        artist_ids, scores = self.implicit_model.recommend(user_id, user_artists_matrix[n], N=n)
        artists = [self.artist_retriever.get_artist_name_from_id(artist_id) for artist_id in artist_ids]
        return artists, scores


if __name__ == "__main__":

    # load user artists matrix
    users = load_user_artists(Path("/content/sample_data/user_artists.dat"))

    # instantiate artist retriever
    artist_retriever = ArtistRetriever()
    artist_retriever.load_artists(Path("/content/sample_data/artists.dat"))

    # instantiate ALS using implicit
    implict_model = implicit.als.AlternatingLeastSquares(
        factors=100, iterations=20, regularization=0.01
    )

    # instantiate recommender, fit, and recommend
    recommender = ImplicitRecommender(artist_retriever, implict_model)
    recommender.fit(users)
    artists, scores = recommender.recommend(6, users, n=50)

    # print results
    for artist, score in zip(artists, scores):
        print(artist)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

Erykah Badu
Toni Braxton
Whitney Houston
Mary J. Blige
Marisa Monte
Bob Marley & The Wailers
Maxwell
Keyshia Cole
Adriana Calcanhotto
Racionais MC's
John Legend
George Michael
Aaliyah
Prince
Corinne Bailey Rae
Jill Scott
Monica
Tamia
Nas
Céline Dion
Ana Carolina
Floetry
Lauryn Hill
Brandy
Aretha Franklin
Tina Turner
Common
Chrisette Michele
Usher
Maria Bethânia
Mariah Carey
Mos Def
Sade
2Pac
Alicia Keys
Amerie
Bryan Adams
Bone Thugs-N-Harmony
Kid Abelha
The-Dream
Janet Jackson
50 Cent
Pharoahe Monch
Boyz II Men
Faith Evans
Kanye West
OutKast
Michael Jackson
Cyndi Lauper
Enigma
